In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-10-06 04:01:39.604378: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-06 04:01:39.604485: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
directory = '/kaggle/input/petfinder-pawpularity-score/train/'
seed = 42
folds = 5

In [3]:
train_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
train_df['img_path'] = directory + train_df['Id'] + '.jpg'
train_df['img_path'].iloc[0]

'/kaggle/input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg'

In [4]:
def split_dataset(train_df, folds = folds, seed = seed):
    x = train_df['img_path']
    y = train_df['Pawpularity']/100.00

    num_bins = int(np.floor(1 + np.log2(len(y))))
    skf = StratifiedKFold(n_splits=folds, shuffle= True, random_state = seed)
    bins = pd.cut(y, bins=num_bins, labels=False)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(x, bins)):
        print(fold, train_idx, val_idx)
        X_train, X_val = x.iloc[train_idx], x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        train_concat = pd.concat((X_train, y_train), axis=1)
        train_concat.to_csv(f'skf_split_train_5cv_fold{fold}.csv')
        val_concat = pd.concat((X_val, y_val), axis = 1)
        val_concat.to_csv(f'skf_split_val_5cv_fold{fold}.csv')

split_dataset(train_df)

0 [   1    3    4 ... 9909 9910 9911] [   0    2   12 ... 9876 9881 9884]
1 [   0    1    2 ... 9908 9909 9911] [   5    6    8 ... 9902 9907 9910]
2 [   0    2    3 ... 9908 9909 9910] [   1   11   21 ... 9905 9906 9911]
3 [   0    1    2 ... 9909 9910 9911] [   3    9   10 ... 9900 9901 9908]
4 [   0    1    2 ... 9908 9910 9911] [   4    7   14 ... 9903 9904 9909]
